# **Ch 15.  보이지 않는 데이터로 하는 딥러닝  :  통합 학습 입문**
------------------------
## **딥러닝의 개인정보 문제**
- 딥러닝은 여러분이 학습 데이터에 접근할 수 있다는 것을 의미합니다.

> 머신러닝의 하위 분야인 딥러닝은 데이터 학습이 전부라고 할 수 있는 만큼 개인적인 정보를 다루는 일이 빈번하다. 딥러닝 모델은 자신을 이해할 수 있도록 하기 위해 타인 수천 명의 개인 정보를 학습할 수 있다는 말이다.
>
> 학습 데이터 없이는 딥러닝이 학습할 수 있는 것은 아무 것도 없다. 딥러닝의 가치를 높에 해주는 용도가 가장 개인적인 데이터셋과 교류하는 것이다 보니, 기업들은 종종 딥러닝 때문에 테이터를 수집하려 한다. 기업이 특정 문제를 해결할려면 개인적인 정보가 불가피하게 필요하기 때문이다.
>
> 그래서 구글에서 모델 학습 때문에 굳이 데이터 셋을 모으지 않아도 되는 기법을 제안했다. "모든 데이터를 한 장소에 모으는 대신, 모델을 데이터 쪽으로 가져가면 어떨까?" 라는 부분에서 탄생된 것이 바로 이번 장에 다룰 **통합 학습(federated learning)**이다.
>
> 이 통합 학습을 이용하면 딥러닝 공급망 사슬에 참여하기 위해 자신의 개인정보를 보낼 필요가 없어진다. 헬스케어, 인사 관리, 기타 다른 민감한 영역의 귀중한 모델들도 사람들에게 개인정보를 공개해달라고 하지 않아도 학습이 가능해진다는 것을 의미한다. 이 기법을 이용해 고객 정보를 공유할 수 없었던 대기업들이 해당 데이터들로 수익을 확보할 수 있게 되었다.

In [0]:
import numpy as np

class Tensor (object):
    
    def __init__(self,data,
                 autograd=False,
                 creators=None,
                 creation_op=None,
                 id=None):
        
        self.data = np.array(data)
        self.autograd = autograd
        self.grad = None

        if(id is None):
            self.id = np.random.randint(0,1000000000)
        else:
            self.id = id
        
        self.creators = creators
        self.creation_op = creation_op
        self.children = {}
        
        if(creators is not None):
            for c in creators:
                if(self.id not in c.children):
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1

    def all_children_grads_accounted_for(self):
        for id,cnt in self.children.items():
            if(cnt != 0):
                return False
        return True 
        
    def backward(self,grad=None, grad_origin=None):
        if(self.autograd):
 
            if(grad is None):
                grad = Tensor(np.ones_like(self.data))

            if(grad_origin is not None):
                if(self.children[grad_origin.id] == 0):
                    return
                    print(self.id)
                    print(self.creation_op)
                    print(len(self.creators))
                    for c in self.creators:
                        print(c.creation_op)
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1

            if(self.grad is None):
                self.grad = grad
            else:
                self.grad += grad
            
            # grads must not have grads of their own
            assert grad.autograd == False
            
            # only continue backpropping if there's something to
            # backprop into and if all gradients (from children)
            # are accounted for override waiting for children if
            # "backprop" was called on this variable directly
            if(self.creators is not None and 
               (self.all_children_grads_accounted_for() or 
                grad_origin is None)):

                if(self.creation_op == "add"):
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad, self)
                    
                if(self.creation_op == "sub"):
                    self.creators[0].backward(Tensor(self.grad.data), self)
                    self.creators[1].backward(Tensor(self.grad.__neg__().data), self)

                if(self.creation_op == "mul"):
                    new = self.grad * self.creators[1]
                    self.creators[0].backward(new , self)
                    new = self.grad * self.creators[0]
                    self.creators[1].backward(new, self)                    
                    
                if(self.creation_op == "mm"):
                    c0 = self.creators[0]
                    c1 = self.creators[1]
                    new = self.grad.mm(c1.transpose())
                    c0.backward(new)
                    new = self.grad.transpose().mm(c0).transpose()
                    c1.backward(new)
                    
                if(self.creation_op == "transpose"):
                    self.creators[0].backward(self.grad.transpose())

                if("sum" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    self.creators[0].backward(self.grad.expand(dim,
                                                               self.creators[0].data.shape[dim]))

                if("expand" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    self.creators[0].backward(self.grad.sum(dim))
                    
                if(self.creation_op == "neg"):
                    self.creators[0].backward(self.grad.__neg__())
                    
                if(self.creation_op == "sigmoid"):
                    ones = Tensor(np.ones_like(self.grad.data))
                    self.creators[0].backward(self.grad * (self * (ones - self)))
                
                if(self.creation_op == "tanh"):
                    ones = Tensor(np.ones_like(self.grad.data))
                    self.creators[0].backward(self.grad * (ones - (self * self)))
                
                if(self.creation_op == "index_select"):
                    new_grad = np.zeros_like(self.creators[0].data)
                    indices_ = self.index_select_indices.data.flatten()
                    grad_ = grad.data.reshape(len(indices_), -1)
                    for i in range(len(indices_)):
                        new_grad[indices_[i]] += grad_[i]
                    self.creators[0].backward(Tensor(new_grad))
                    
                if(self.creation_op == "cross_entropy"):
                    dx = self.softmax_output - self.target_dist
                    self.creators[0].backward(Tensor(dx))
                    
    def __add__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data + other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="add")
        return Tensor(self.data + other.data)

    def __neg__(self):
        if(self.autograd):
            return Tensor(self.data * -1,
                          autograd=True,
                          creators=[self],
                          creation_op="neg")
        return Tensor(self.data * -1)
    
    def __sub__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data - other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="sub")
        return Tensor(self.data - other.data)
    
    def __mul__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data * other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="mul")
        return Tensor(self.data * other.data)    

    def sum(self, dim):
        if(self.autograd):
            return Tensor(self.data.sum(dim),
                          autograd=True,
                          creators=[self],
                          creation_op="sum_"+str(dim))
        return Tensor(self.data.sum(dim))
    
    def expand(self, dim,copies):

        trans_cmd = list(range(0,len(self.data.shape)))
        trans_cmd.insert(dim,len(self.data.shape))
        new_data = self.data.repeat(copies).reshape(list(self.data.shape) + [copies]).transpose(trans_cmd)
        
        if(self.autograd):
            return Tensor(new_data,
                          autograd=True,
                          creators=[self],
                          creation_op="expand_"+str(dim))
        return Tensor(new_data)
    
    def transpose(self):
        if(self.autograd):
            return Tensor(self.data.transpose(),
                          autograd=True,
                          creators=[self],
                          creation_op="transpose")
        
        return Tensor(self.data.transpose())
    
    def mm(self, x):
        if(self.autograd):
            return Tensor(self.data.dot(x.data),
                          autograd=True,
                          creators=[self,x],
                          creation_op="mm")
        return Tensor(self.data.dot(x.data))
    
    def sigmoid(self):
        if(self.autograd):
            return Tensor(1 / (1 + np.exp(-self.data)),
                          autograd=True,
                          creators=[self],
                          creation_op="sigmoid")
        return Tensor(1 / (1 + np.exp(-self.data)))

    def tanh(self):
        if(self.autograd):
            return Tensor(np.tanh(self.data),
                          autograd=True,
                          creators=[self],
                          creation_op="tanh")
        return Tensor(np.tanh(self.data))
    
    def index_select(self, indices):

        if(self.autograd):
            new = Tensor(self.data[indices.data],
                         autograd=True,
                         creators=[self],
                         creation_op="index_select")
            new.index_select_indices = indices
            return new
        return Tensor(self.data[indices.data])
    
    def softmax(self):
        temp = np.exp(self.data)
        softmax_output = temp / np.sum(temp,
                                       axis=len(self.data.shape)-1,
                                       keepdims=True)
        return softmax_output
    
    def cross_entropy(self, target_indices):

        temp = np.exp(self.data)
        softmax_output = temp / np.sum(temp,
                                       axis=len(self.data.shape)-1,
                                       keepdims=True)
        
        t = target_indices.data.flatten()
        p = softmax_output.reshape(len(t),-1)
        target_dist = np.eye(p.shape[1])[t]
        loss = -(np.log(p) * (target_dist)).sum(1).mean()
    
        if(self.autograd):
            out = Tensor(loss,
                         autograd=True,
                         creators=[self],
                         creation_op="cross_entropy")
            out.softmax_output = softmax_output
            out.target_dist = target_dist
            return out

        return Tensor(loss)
        
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())  

class Layer(object):
    
    def __init__(self):
        self.parameters = list()
        
    def get_parameters(self):
        return self.parameters

    
class SGD(object):
    
    def __init__(self, parameters, alpha=0.1):
        self.parameters = parameters
        self.alpha = alpha
    
    def zero(self):
        for p in self.parameters:
            p.grad.data *= 0
        
    def step(self, zero=True):
        
        for p in self.parameters:
            
            p.data -= p.grad.data * self.alpha
            
            if(zero):
                p.grad.data *= 0


class Linear(Layer):

    def __init__(self, n_inputs, n_outputs, bias=True):
        super().__init__()
        
        self.use_bias = bias
        
        W = np.random.randn(n_inputs, n_outputs) * np.sqrt(2.0/(n_inputs))
        self.weight = Tensor(W, autograd=True)
        if(self.use_bias):
            self.bias = Tensor(np.zeros(n_outputs), autograd=True)
        
        self.parameters.append(self.weight)
        
        if(self.use_bias):        
            self.parameters.append(self.bias)

    def forward(self, input):
        if(self.use_bias):
            return input.mm(self.weight)+self.bias.expand(0,len(input.data))
        return input.mm(self.weight)


class Sequential(Layer):
    
    def __init__(self, layers=list()):
        super().__init__()
        
        self.layers = layers
    
    def add(self, layer):
        self.layers.append(layer)
        
    def forward(self, input):
        for layer in self.layers:
            input = layer.forward(input)
        return input
    
    def get_parameters(self):
        params = list()
        for l in self.layers:
            params += l.get_parameters()
        return params


class Embedding(Layer):
    
    def __init__(self, vocab_size, dim):
        super().__init__()
        
        self.vocab_size = vocab_size
        self.dim = dim
        
        # this random initialiation style is just a convention from word2vec
        self.weight = Tensor((np.random.rand(vocab_size, dim) - 0.5) / dim, autograd=True)
        
        self.parameters.append(self.weight)
    
    def forward(self, input):
        return self.weight.index_select(input)


class Tanh(Layer):
    def __init__(self):
        super().__init__()
    
    def forward(self, input):
        return input.tanh()


class Sigmoid(Layer):
    def __init__(self):
        super().__init__()
    
    def forward(self, input):
        return input.sigmoid()
    

class CrossEntropyLoss(object):
    
    def __init__(self):
        super().__init__()
    
    def forward(self, input, target):
        return input.cross_entropy(target)

class MSELoss(object):
    
    def __init__(self):
        super().__init__()
    
    def forward(self, input, target):
        dif = input - target
        return (dif * dif).sum(0)
    
class RNNCell(Layer):
    
    def __init__(self, n_inputs, n_hidden, n_output, activation='sigmoid'):
        super().__init__()

        self.n_inputs = n_inputs
        self.n_hidden = n_hidden
        self.n_output = n_output
        
        if(activation == 'sigmoid'):
            self.activation = Sigmoid()
        elif(activation == 'tanh'):
            self.activation == Tanh()
        else:
            raise Exception("Non-linearity not found")

        self.w_ih = Linear(n_inputs, n_hidden)
        self.w_hh = Linear(n_hidden, n_hidden)
        self.w_ho = Linear(n_hidden, n_output)
        
        self.parameters += self.w_ih.get_parameters()
        self.parameters += self.w_hh.get_parameters()
        self.parameters += self.w_ho.get_parameters()        
    
    def forward(self, input, hidden):
        from_prev_hidden = self.w_hh.forward(hidden)
        combined = self.w_ih.forward(input) + from_prev_hidden
        new_hidden = self.activation.forward(combined)
        output = self.w_ho.forward(new_hidden)
        return output, new_hidden
    
    def init_hidden(self, batch_size=1):
        return Tensor(np.zeros((batch_size,self.n_hidden)), autograd=True)
    
class LSTMCell(Layer):
    
    def __init__(self, n_inputs, n_hidden, n_output):
        super().__init__()

        self.n_inputs = n_inputs
        self.n_hidden = n_hidden
        self.n_output = n_output

        self.xf = Linear(n_inputs, n_hidden)
        self.xi = Linear(n_inputs, n_hidden)
        self.xo = Linear(n_inputs, n_hidden)        
        self.xc = Linear(n_inputs, n_hidden)        
        
        self.hf = Linear(n_hidden, n_hidden, bias=False)
        self.hi = Linear(n_hidden, n_hidden, bias=False)
        self.ho = Linear(n_hidden, n_hidden, bias=False)
        self.hc = Linear(n_hidden, n_hidden, bias=False)        
        
        self.w_ho = Linear(n_hidden, n_output, bias=False)
        
        self.parameters += self.xf.get_parameters()
        self.parameters += self.xi.get_parameters()
        self.parameters += self.xo.get_parameters()
        self.parameters += self.xc.get_parameters()

        self.parameters += self.hf.get_parameters()
        self.parameters += self.hi.get_parameters()        
        self.parameters += self.ho.get_parameters()        
        self.parameters += self.hc.get_parameters()                
        
        self.parameters += self.w_ho.get_parameters()        
    
    def forward(self, input, hidden):
        
        prev_hidden = hidden[0]        
        prev_cell = hidden[1]
        
        f = (self.xf.forward(input) + self.hf.forward(prev_hidden)).sigmoid()
        i = (self.xi.forward(input) + self.hi.forward(prev_hidden)).sigmoid()
        o = (self.xo.forward(input) + self.ho.forward(prev_hidden)).sigmoid()        
        g = (self.xc.forward(input) + self.hc.forward(prev_hidden)).tanh()        
        c = (f * prev_cell) + (i * g)

        h = o * c.tanh()
        
        output = self.w_ho.forward(h)
        return output, (h, c)
    
    def init_hidden(self, batch_size=1):
        init_hidden = Tensor(np.zeros((batch_size,self.n_hidden)), autograd=True)
        init_cell = Tensor(np.zeros((batch_size,self.n_hidden)), autograd=True)
        init_hidden.data[:,0] += 1
        init_cell.data[:,0] += 1
        return (init_hidden, init_cell)

## **통합 학습**

- 학습이 목적이라면 데이터셋에 대한 접근 권한이 필요하진 않습니다.

> 통합 학습은 보안 환경으로 들어가서 데이터를 옮기지 않고도 문제를 해결하는 방법을 학습하는 모델에 관한 것이다. 코드를 살펴보자.

In [0]:
import numpy as np
from collections import Counter
import random
import sys
np.random.seed(12345)

import codecs
with codecs.open('spam.txt', "r",encoding='utf-8', errors='ignore') as fdata:
    raw = fdata.readlines()

vocab = set()
    
spam = list()
for row in raw:
    spam.append(set(row[:-2].split(" ")))
    for word in spam[-1]:
        vocab.add(word)
    
import codecs
with codecs.open('ham.txt', "r",encoding='utf-8', errors='ignore') as fdata:
    raw = fdata.readlines()

ham = list()
for row in raw:
    ham.append(set(row[:-2].split(" ")))
    for word in ham[-1]:
        vocab.add(word)
        
vocab.add("<unk>")

vocab = list(vocab)
w2i = {}
for i,w in enumerate(vocab):
    w2i[w] = i
    
def to_indices(input, l=500):
    indices = list()
    for line in input:
        if(len(line) < l):
            line = list(line) + ["<unk>"] * (l - len(line))
            idxs = list()
            for word in line:
                idxs.append(w2i[word])
            indices.append(idxs)
    return indices

## **스팸 탐지 학습**
- 실제 사용자의 이메일을 이용해서 스팸 탐지 모델을 학습하고 싶다고 합시다.

> 첫 번째 모델은 엔론 데이터셋(유명한 엔론 소송을 통해 알려진 대형 이메일 말뭉치)이라고 하는 공개 데이터셋을 이용해서 학습한다. 이 데이터셋에는 여러 가지 개인적인 정보가 담겨있는데 법정 소송을 통해 대중에게 모두 공개되었다고 한다.
>
> 앞 코드와 이번 코드는 전처리(preprocessin)만 수행한다. 13장에서 딥러닝 프레임워크를 만들 때 생성했던 임베딩 클래스로 순전파를 넣을 수 있도록 전처리 과정이 필수적이다. 앞서 수행한 코드와 같이 이 말뭉치 속의 모든 단어는 인덱스된 목록으로 변환하는 과정을 거친다. 그리고 모든 이메일을 추리거나 `<unk>` 토큰으로 채워 넣어서 정확히 500개의 단어로만 구성되도록 바꿔야한다. 그래서 최종 데이터 셋은 결과적으로 **정방형**이 된다.
>
> 참고 ) unk는 unknown token의 줄임말이다.

In [0]:
spam_idx = to_indices(spam)
ham_idx = to_indices(ham)

train_spam_idx = spam_idx[0:-1000]
train_ham_idx = ham_idx[0:-1000]

test_spam_idx = spam_idx[-1000:]
test_ham_idx = ham_idx[-1000:]

train_data = list()
train_target = list()

test_data = list()
test_target = list()

for i in range(max(len(train_spam_idx),len(train_ham_idx))):
    train_data.append(train_spam_idx[i%len(train_spam_idx)])
    train_target.append([1])
    
    train_data.append(train_ham_idx[i%len(train_ham_idx)])
    train_target.append([0])
    
for i in range(max(len(test_spam_idx),len(test_ham_idx))):
    test_data.append(test_spam_idx[i%len(test_spam_idx)])
    test_target.append([1])
    
    test_data.append(test_ham_idx[i%len(test_ham_idx)])
    test_target.append([0])

In [0]:
def train(model, input_data, target_data, batch_size=500, iterations=5):
    
    criterion = MSELoss()
    optim = SGD(parameters=model.get_parameters(), alpha=0.01)
    
    n_batches = int(len(input_data) / batch_size)
    bs = batch_size
    for iter in range(iterations):
        iter_loss = 0
        for b_i in range(n_batches):

            # padding token should stay at 0
            model.weight.data[w2i['<unk>']] *= 0 
            input = Tensor(input_data[b_i*bs:(b_i+1)*bs], autograd=True)
            target = Tensor(target_data[b_i*bs:(b_i+1)*bs], autograd=True)

            pred = model.forward(input).sum(1).sigmoid()
            loss = criterion.forward(pred,target)
            loss.backward()
            optim.step()

            iter_loss += loss.data[0] / bs

            sys.stdout.write("\r\tLoss:" + str(iter_loss / (b_i+1)))
        print()
    return model

In [0]:
def test(model, test_input, test_output):
    
    model.weight.data[w2i['<unk>']] *= 0 
    
    input = Tensor(test_input, autograd=True)
    target = Tensor(test_output, autograd=True)

    pred = model.forward(input).sum(1).sigmoid()
    return ((pred.data > 0.5) == target.data).mean()

위와 같이  train()과 test() 함수를 이용하면 이어지는 코드를 이용해 신경망을 초기화하고 학습시킬 수 있다. 테스트 데이터셋의 균형이 잡혀있는 상태라 세 번만 돌려도 99% 정도의 정확도로 분류를 수행할 수 있다.

In [0]:
model = Embedding(vocab_size=len(vocab), dim=1)
model.weight.data *= 0
criterion = MSELoss()
optim = SGD(parameters=model.get_parameters(), alpha=0.01)

In [19]:
for i in range(3):
    model = train(model, train_data, train_target, iterations=1)
    print("% Correct on Test Set: " + str(test(model, test_data, test_target)*100))

	Loss:0.04895785149340228
% Correct on Test Set: 98.75
	Loss:0.014751484575885144
% Correct on Test Set: 99.15
	Loss:0.01073762530961579
% Correct on Test Set: 99.2


## **통합해봅시다**

- 앞 예제 코드는 평범한 바닐라 딥러닝이었습니다. 이제 개인정보를 보호해볼까요?

앞선 예제는 모든 이메일을 한 곳에 모아서 딥러닝을 돌렸다. 이번에는 여러 개의 다양한 이메일 컬렉션을 가지는 **통합 학습 환경**을 시뮬레이션해보자

In [0]:
bob = (train_data[0:1000], train_target[0:1000])
alice = (train_data[1000:2000], train_target[1000:2000])
sue = (train_data[2000:], train_target[2000:])

위 코드는 이전과 동일한 학습을 할 수 있지만, 이번에는 모든 사람의 이메일 데이터베이스를 동시에 처리한다. 각 반복을 수행한 후에, Bob, Alice, Sue로부터 나온 모델 값에 대해 평균을 내고 평가한다. 참고로 통합 학습의 어떤 파생 기법들은 각 배치 후에 합치기도 한다. 여기서는 간단히 처리하였다.

In [0]:
model = Embedding(vocab_size=len(vocab), dim=1)
model.weight.data *= 0

In [24]:
import copy

for i in range(3):
    print("Starting Training Round...")
    print("\tStep 1: send the model to Bob")
    bob_model = train(copy.deepcopy(model), bob[0], bob[1], iterations=1)
    
    print("\n\tStep 2: send the model to Alice")
    alice_model = train(copy.deepcopy(model), alice[0], alice[1], iterations=1)
    
    print("\n\tStep 3: Send the model to Sue")
    sue_model = train(copy.deepcopy(model), sue[0], sue[1], iterations=1)
    
    print("\n\tAverage Everyone's New Models")
    model.weight.data = (bob_model.weight.data + \
                         alice_model.weight.data + \
                         sue_model.weight.data)/3
    
    print("\t% Correct on Test Set: " + \
          str(test(model, test_data, test_target)*100))
    
    print("\nRepeat!!\n")

Starting Training Round...
	Step 1: send the model to Bob
	Loss:0.21908166249699718

	Step 2: send the model to Alice
	Loss:0.2937106899184867

	Step 3: Send the model to Sue
	Loss:0.04370172144365995

	Average Everyone's New Models
	% Correct on Test Set: 87.0

Repeat!!

Starting Training Round...
	Step 1: send the model to Bob
	Loss:0.07604929485450417

	Step 2: send the model to Alice
	Loss:0.10877523800360037

	Step 3: Send the model to Sue
	Loss:0.025948541543226986

	Average Everyone's New Models
	% Correct on Test Set: 92.7

Repeat!!

Starting Training Round...
	Step 1: send the model to Bob
	Loss:0.03756810302218815

	Step 2: send the model to Alice
	Loss:0.04502773446215378

	Step 3: Send the model to Sue
	Loss:0.019466797770650156

	Average Everyone's New Models
	% Correct on Test Set: 98.75

Repeat!!



위 코드에서는 예전 모델과 거의 동일한  성능을 보여주고 있지만 접근 권한이 있는지에 대해서는 모호성을 띠고 있다. 이 사람들의 데이터셋에 대한 정보를 전혀 알아낼 수 없을까? 

## **통합 학습 해킹하기**

- 장난감 예제를 이용해 데이터셋을 학습하는 방법을 관찰해봅시다.

> 통합 학습에는 두 가지 문제점이 있다. 이 문제 모두 학습 데이터셋 안에 각 갠인이 소량의 데이터 예제만 가지고 있을 경우에만 발생한다. 이는 성능과 개인정보 보호에 관련된 것인데, 누군가가 소수의 학습 예제만 갖고 있는 경우에는 데이터에 대해 많은 것을 파악할 수 있는 것으로 드러났다. 게다가 예를 들어 10,000명의 데이터(각각이 소수 정보를 제공)가 제공되면, (특히 모델을 교환하는 데 대부분의 시간을 사용하고 학습에는 시간을 별로 할애하지 못하는 문제점을 갖고 있었다.
>
> 사용자가 단일 배치상에서 가중치 갱신을 수행할 때 무엇을 학습할 수 있는지 살펴보자. 


In [25]:
import copy

bobs_email = ["my", "computer", "password", "is", "pizza"]

bob_input = np.array([[w2i[x] for x in bobs_email]])
bob_target = np.array([[0]])

model = Embedding(vocab_size=len(vocab), dim=1)
model.weight.data *= 0

bobs_model = train(copy.deepcopy(model), bob_input, bob_target, iterations=1, batch_size=1)

	Loss:0.25


>Bob은 받은 편지함 안에 있는 이메일을 이용해 모델을 생성하고 갱신하려 한다. 그런데 Bob은 암호를 이메일 안에 저장해뒀다.
>
>이메일 내용은 "My computer passward is pizza"이다. 이제 어떤 가중치가 바뀌었는지 살펴보면 Bob이 받은 이메일의 어휘를 파악하거나 추론할 수 있다.

In [26]:
for i, v in enumerate(bobs_model.weight.data - model.weight.data):
    if(v != 0):
        print(vocab[i])

computer
pizza
password
my
is


>위와 같이 이메일 내용을 알아냈다. 학습 데이터셋이 가중치 갱신 과정에서 얻은 정보가 무엇인지 쉽게 알아낼 수 있다면 통합 학습을 제대로 사용할 수 있을까?
>


## **보안 통합**
-누가 보기 전에 방대한 개인 데이터에 대한 가중치 갱신값을 평균화하세요.
 > 이 문제의 해결책은 Bob이 절대로 경사도를 공개된 곳에 두지 않도록 하는 것이다. 하지만 사람들이 보지 못하도록 해놓은 자신만의 경사도에 Bob이 기여할 수 있는 방법은 무엇일까? 이는 사회 과학에서 사용하는 **무작위 응답(randomized response)** 기법을 잠깐 살펴보자.
>
> 예시로 100명에게 범죄를 저질렀는지를 물어보는 설문을 진행하는 것을 들고 있다. 이는 모든 대답이 "아니오"로 나올 것이다. 설문 결과를 발설하지 않겠다고 약속을 해도 결과는 같을 것이다. 하지만 동전을 두 번 던져서 처음 던졌을 때 숫자가 나오는 경우에만 피설문자가 정직하게 응답해야 한다고 해보자. 그림이 나왔을 때는 두 번째 던진 결과에 따라 "예" 또는 "아니오"라고 답하면 된다.
>
> 위 실험을 진행하면 진짜 응답은 첫 번째 동전 던지기와 두 번째 동전던지기의 무작위적인 잡음 뒤에 숨게 된다. 만약 응답자의 60%가 "예"라고 답한다면, 약 70%의 응답자가 범죄를 저질렀다고 판단할 수 있다. 그러니까 어떤 사람에 관한 정보가 그 사람이 아닌 잡음으로부터 나올 수 있음을 무작위적 잡음이 합리화시킨다는 점이 이 발상의 요지다.
>
> - "그럴듯한 부인"을 통한 개인정보 보호 / 개인이 아닌 무작위적 잡음으로부터 특정 답변이 나올 가능성이 '**그럴듯한 부인(plausible deniability)**'이라는 명분을 제공함으로써 사람들의 개인정보를 보호해준다. 이는 보안 통합과 더불어 차별적인 개인정보 보호를 위한 기반을 형성한다.
>
> 자신의 경사도를 제외한 다른 사람의 볼 수 없도록 하는 방식으로 모든 참가자의 경사도를 종합하는 것이 좋다. 이런 문제를 다루는 과업을 일컬어 **보안 통합(secureaggregation)**이라고 하며, **동형 암호화(homomorphic encryption)**라는 도구를 사용해야 한다.

## **동형 암호화**
- 암호화된 값도 수리 연산이 가능합니다.
> 가장 흥미로운 미개척 연구 분야는 딥러닝을 포함한 인공지능과 암호학, 이 두 분야의 교집합 분야다. 이 교집합 분야의 전면과 중심에는 동형 암호화라고 하는 것이 있는데 이는 쉽게 말해 암호화된 값을 복호화하지 않은 상태에서 계산이 가능하도록 하는 기술이다.
>
> 특히, 암호화된 값 사이의 덧셈이 필요한데 이 책에서는 동형 암호화의 몇 가지 정의와 함께 어떻게 동작하는 지 정도만 보여주고 있다.
>
> 미리 숙지해야 할 개념!
> 
> 공개 키(public key) : 숫자를 암호화할 때 사용한다.
> 개인 키(private key) : 암호화된 숫자를 복호화할 때 사용한다.
> 암호문(ciphertext) : 암호화된 값
> 평문(plaintext) : 암호화되지 않은 값

> 아래 코드는 phe 라이브러리를 이용한 동형 암호와 예제이다. 

In [32]:
pip install phe

  Created wheel for phe: filename=phe-1.4.0-py2.py3-none-any.whl size=37362 sha256=49c492981a5d788552d86aac4a83407ba10965db7559da610532ecda3cefddd4
  Stored in directory: /root/.cache/pip/wheels/f8/dc/36/dcb6bf0f1b9907e7b710ace63e64d08e7022340909315fdea4
Successfully built phe


In [33]:
import phe

public_key, private_key = phe.generate_paillier_keypair(n_length=1024)

# encrypt the number "5"
x = public_key.encrypt(5)

# encrypt the number "3"
y = public_key.encrypt(3)

# add the two encrypted values
z = x + y

# decrypt the result
z_ = private_key.decrypt(z)
print("The Answer: " + str(z_))

The Answer: 8


>위 코드는 두 수, 5와 3을 암호화 한 뒤, 암호화 상태에서 두 수를 더한다. 동형 암호화와 유사한 기능을 구현하는 것이 있는데 바로 **보안 다중 계산(secure mulitipart computation)**이다. 이는 다루지 않는다.
>
> 다시 보안 통합 문제로 돌아가면 볼 수 없는 수를 더할 수 있다는 점을 고려하게 되면 간단하게 수행할 수 있다. 모델을 초기화하는 사람은 공개 키를 Bob, Alice, Sue에게 보내 그들의 가중치를 암호화하도록 한다. 그다음, 개인 키를 가지고 있지 않은 Bob, Alice, Sue는 서로 교류하며 자신들의 경사도를 하나의 최종 갱신값으로 축적한 뒤에 개인 키를 이용해 복호화할 수 있는 권한을 가진 모델 소유자에게 보낸다.

## **동형 암호화 통합 학습**
- 통합되는 경사도 보호를 위해 동형 암호화를 이용해보자.

In [0]:
model = Embedding(vocab_size=len(vocab), dim=1)
model.weight.data *= 0

# note that in production the n_length should be at least 1024
public_key, private_key = phe.generate_paillier_keypair(n_length=128)

def train_and_encrypt(model, input, target, pubkey):
    new_model = train(copy.deepcopy(model), input, target, iterations=1)

    encrypted_weights = list()
    for val in new_model.weight.data[:,0]:
        encrypted_weights.append(public_key.encrypt(val))
    ew = np.array(encrypted_weights).reshape(new_model.weight.data.shape)
    
    return ew

In [35]:
for i in range(3):
    print("\nStarting Training Round...")
    print("\tStep 1: send the model to Bob")
    bob_encrypted_model = train_and_encrypt(copy.deepcopy(model), 
                                            bob[0], bob[1], public_key)

    print("\n\tStep 2: send the model to Alice")
    alice_encrypted_model = train_and_encrypt(copy.deepcopy(model), 
                                              alice[0], alice[1], public_key)

    print("\n\tStep 3: Send the model to Sue")
    sue_encrypted_model = train_and_encrypt(copy.deepcopy(model), 
                                            sue[0], sue[1], public_key)

    print("\n\tStep 4: Bob, Alice, and Sue send their")
    print("\tencrypted models to each other.")
    aggregated_model = bob_encrypted_model + \
                       alice_encrypted_model + \
                       sue_encrypted_model

    print("\n\tStep 5: only the aggregated model")
    print("\tis sent back to the model owner who")
    print("\t can decrypt it.")
    raw_values = list()
    for val in sue_encrypted_model.flatten():
        raw_values.append(private_key.decrypt(val))
    model.weight.data = np.array(raw_values).reshape(model.weight.data.shape)/3

    print("\t% Correct on Test Set: " + \
              str(test(model, test_data, test_target)*100))


Starting Training Round...
	Step 1: send the model to Bob
	Loss:0.21908166249699718

	Step 2: send the model to Alice
	Loss:0.2937106899184867

	Step 3: Send the model to Sue
	Loss:0.04370172144365995

	Step 4: Bob, Alice, and Sue send their
	encrypted models to each other.

	Step 5: only the aggregated model
	is sent back to the model owner who
	 can decrypt it.
	% Correct on Test Set: 98.75

Starting Training Round...
	Step 1: send the model to Bob
	Loss:0.06612036480821967

	Step 2: send the model to Alice
	Loss:0.0680059577116942

	Step 3: Send the model to Sue
	Loss:0.03436716023928057

	Step 4: Bob, Alice, and Sue send their
	encrypted models to each other.

	Step 5: only the aggregated model
	is sent back to the model owner who
	 can decrypt it.
	% Correct on Test Set: 98.9

Starting Training Round...
	Step 1: send the model to Bob
	Loss:0.06227552561089021

	Step 2: send the model to Alice
	Loss:0.06738138277451584

	Step 3: Send the model to Sue
	Loss:0.03374911228415595

	St

> 이제 새로운 학습 계획을 실행할 수 있게 되었다. 이 학습 계획은 예전에 비해 한 가지 단계가 더 추가되었다. Alice, Bob, Sue는 자신들의 동형 암호화 모델을 반환하기에 앞서 이들을 더한다, 그 때문에 **누구도 어떤 사람의 어떤 경사도가 갱신되었는지 알 수 없다.**('그럴 듯한 부인'의 한 형태이다.) 실전에서는 어떤 무작위 noise를 추가하는 것만으로도 Bob. Alice, Sue의 개인정보를 어느 정도 보호할 수 있는 수준을 충족할 수 있다. 

## **요약**

- 통합 학습은 딥러닝에서 가장 흥미진진한 약진 중 하나이다.
> 통합 학습을 이용하므로써 예전에는 손댈 수 없을 정도로 민감했던 데이터셋을 새롭게 사용할 수 있을 것이다. 인공지능과 암호화 연구 사이의 교집합, 즉 폭넓은 융합인 통합 학습이 최근 10년 동안 나타났던 가장 흥미진진한 융합 영역이다.
>
> 통합 학습을 실전에서 사용할 수 없게 하는 주요 장애물은 현대 딥러닝 toolkit에서 이 기술을 지원하지 않는다는 것이다. pip install.. 을 실행해서 통합 학습을 사용할 수 있을 때가 tipping point가 될 것이다. 그때는 개인정보와 보안이 이들 시민이며, 통합 학습, 동형 암호화, 차등적 개인정보 보호, 보안 다중 계산 기능이 내장된 딥러닝 framework를 사용할 수 있을 것이다.
